# Looking at age in the KPM results

## Authors
- **David W. Hogg** (NYU) (MPIA) (Flatiron)
- **Emily J. Griffith** (Colorado)



In [ ]:
import numpy as np
import pylab as plt
from astropy.table import Table

In [ ]:
# read and cut table
foo = Table.read("../data/APOK_david.csv.gz", format="csv")
good = np.logical_and((foo["MG_H_pred"] > -0.50), (foo["LogAge"] > 2.00))
foo = foo[good]

In [ ]:
print(foo.columns)

In [ ]:
foo["FE_MG_obs"] = foo["FE_H_obs"] - foo["MG_H_obs"]
print(foo.columns)

## Look at residuals away from the KPM vs age

In [ ]:
cols = list(foo.columns)
for col in cols:
    if col[-4:] == "_obs":
        predcol = col[:-4] + "_pred"
        newcol = col[:-4] + "_resid"
        if predcol in cols:
            print("making", newcol)
            foo[newcol] = foo[col] - foo[predcol]
print(foo.columns)

In [ ]:
labeldict = {}
for col in foo.columns:
    bar = col.split("_")
    if len(bar) == 3:
        if len(bar[0]) == 2:
            bar[0] = bar[0][0] + bar[0][1].lower()
        if bar[0] == "C+N":
            bar[0] = "(C+N)"
        if len(bar[1]) == 2:
            bar[1] = bar[1][0] + bar[1][1].lower()
        label = "[" + bar[0] + "/" + bar[1] + "]"
        if bar[2] == "obs":
            label = "observed " + label
        if bar[2] == "pred":
            label = "KPM-predicted " + label
        if bar[2] == "resid":
            label = label + " residual"
        labeldict[col] = label
print(labeldict)

In [ ]:
for xcol, xlabel, (a, b) in [("LogAge", "log age", ( 2.4, 4.3)),
                             ("C_N",    "[C/N]",   (-1.0, 0.3))]:
    X = np.concatenate((foo[xcol][:, None], np.ones_like(foo[xcol])[:, None]), axis=1)
    j = 0
    fig, ax = plt.subplots(4, 4, figsize=(14, 14))
    ax = ax.flatten()
    for col in foo.columns:
        if col[-6:] == "_resid":
            (slope, inter), _, _, _ = np.linalg.lstsq(X, foo[col], rcond=None)
            ax[j].scatter(foo[xcol], foo[col], s=1, c=(foo["AIa"] / foo["Acc"]))
            ax[j].axhline(0., color="k", lw=1, alpha=0.25, zorder=99)
            ax[j].plot([a, b], [slope * a + inter, slope * b + inter], "k-", lw=1, zorder=100)
            ax[j].set_xlim(a, b)
            ax[j].set_ylim(-0.4, 0.4)
            ax[j].text(a, 0.34, " " + labeldict[col] + " vs " + xlabel)
            j += 1
    for jj in range(j, len(ax)):
        ax[jj].set_axis_off()
    fig.savefig("resids_vs_" + xcol + ".png")

## Look at inferring ages from amplitudes and [C/N]

In [ ]:
plt.scatter(np.log(foo["Acc"]), foo["AIa"] / foo["Acc"], s=1, c=foo["LogAge"])
plt.xlabel("ln(Acc)")
plt.ylabel("AIa / Acc")
plt.colorbar(label="log age")
plt.title("sample")

In [ ]:
plt.scatter(foo["C_N"], foo["AIa"] / foo["Acc"], s=1, c=foo["LogAge"])
plt.xlabel("[C/N]")
plt.ylabel("AIa / Acc")
plt.colorbar(label="log age")
plt.title("sample")

In [ ]:
# make features and labels
X = np.zeros((len(foo), 3))
X[:, 0] = np.log(foo["Acc"])
X[:, 1] = np.log(foo["AIa"])
X[:, 2] = foo["C_N"]
Y = np.zeros((len(foo), 2))
Y[:, 0] = foo["LogAge"]
Y[:, 1] = foo["Numax"]
print(X.shape, Y.shape)

In [ ]:
def knn_regression(X, Y, K, metric=None, dist2=None):
    """
    This is the most naive possible implementation.

    ## Bugs:
    - A tiny bit of experimenting said `np.mean()` is better than `np.median()` but this is uncertain.
    - Not `NaN` safe.
    - `metric` is just the diagonal entries of the metric.
    - Computes all distances when it should use a tree or something when `N` is large.
    """
    N, D = X.shape
    en, M = Y.shape
    assert en == N
    if metric is None:
        metric = np.ones_like(X[0])
    if dist2 is None:
        dist2 = np.sum(metric[None, None, :] * (X[:, None, :] - X[None, :, :]) ** 2, axis=-1)
    indxs = np.argsort(dist2, axis=-1)[:, 1:K+1]
    return np.mean(Y[indxs], axis=1), dist2

In [ ]:
# make metric and regress
metric = np.array([1., 1., 1.])
K = 41
Yhat, _ = knn_regression(X, Y, K, metric)
print(Yhat.shape)

In [ ]:
plt.scatter(Y[:, 0], Yhat[:, 0], s=1, c=foo["C_N"])
a, b = plt.ylim()
plt.plot([a, b], [a, b], "k-", alpha=0.5)
plt.ylim(a, b)
plt.xlabel("true label")
plt.ylabel("predicted label")
plt.colorbar(label="[C/N]")
plt.title("log age")

In [ ]:
# now do a hyper-parameter searchd
metric = np.array([1., 1., 1.])
for K in np.arange(35, 50, 2):
    Yhat, _ = knn_regression(X, Y, K, metric)
    mad = np.median(np.abs(Y[:, 0] - Yhat[:, 0])) # just look at logage
    print(K, mad)

In [ ]:
# now do another hyper-parameter search
K = 41
metric = np.array([1., 1., 1.])
for alpha in [0.3, 0.6, 1.0, 1.5, 3.0]:
    metric[-1] = alpha
    Yhat, _ = knn_regression(X, Y, K, metric)
    mad = np.median(np.abs(Y[:, 0] - Yhat[:, 0])) # just look at logage
    print(metric, mad)